In [1]:
import os
import pandas as pd
import numpy as np
import datetime as dt
from fredapi import Fred

In [2]:
key = ""
with open(f"../assets/fred.key", "r") as file:
    key = file.read()

fred = Fred(api_key = key)

In [3]:
features = "HCOMPBS OPHPBS PCEPI CPIAUCSL IPDBS".split()

fred_df = pd.DataFrame()
for f in features:
    series = fred.get_series(f)
    series.name = f

    fred_df[f] = series

fred_df


,HCOMPBS,OPHPBS,PCEPI,CPIAUCSL,IPDBS
1947-01-01,3.464,20.351,NaN,21.480,14.646
1947-04-01,3.565,20.440,NaN,22.000,14.847
1947-07-01,3.618,20.235,NaN,22.230,15.240
1947-10-01,3.751,20.502,NaN,22.910,15.662
1948-01-01,3.802,20.981,NaN,23.680,15.775
...,...,...,...,...,...
2019-10-01,121.051,108.729,110.377,257.387,110.786
2020-01-01,123.643,108.619,110.917,258.687,111.004
2020-04-01,130.359,111.257,110.131,256.192,109.787
2020-07-01,128.764,112.750,111.152,258.604,111.027


In [17]:
cps_df = pd.read_stata("../data/earnings.dta")
cps_df = cps_df[[
    "year",
    "month",
    "incwage",
    "incbus",
    "incfarm"
]]

cps_df.head()

,year,month,incwage,incbus,incfarm
0,1962,march,0.0,0.0,0.0
1,1962,march,99999999.0,99999999.0,99999999.0
2,1962,march,99999999.0,99999999.0,99999999.0
3,1962,march,1692.0,0.0,0.0
4,1962,march,1522.0,0.0,0.0


In [18]:
wage_cols = [
    "incwage",
    "incbus",
    "incfarm"
]

# Set top-codes to 0
for c in wage_cols:
    cps_df[c] = cps_df[c].replace(99999999, np.nan)
    cps_df[c] = cps_df[c].replace(99999998, np.nan)

cps_df = cps_df.dropna()

# Sum the total income for each individual
cps_df["inctot"] = np.sum(cps_df[wage_cols], axis=1)
cps_df = cps_df[["year", "month", "inctot"]]

cps_df.head()

,year,month,inctot
0,1962,march,0.0
3,1962,march,1692.0
4,1962,march,1522.0
5,1962,march,1500.0
6,1962,march,2300.0


In [19]:
qmap = {
    "january":   1,
    "february":  1,
    "march":     1,
    "april":     4,
    "may":       4,
    "june":      4,
    "july":      7,
    "august":    7,
    "september": 7,
    "october":   10,
    "november":  10,
    "december": 10,
}

cps_df["month"] = cps_df["month"].map(qmap)
cps_df["date"]  = cps_df["year"].astype(str) + '-' + cps_df["month"].astype(str) + "-1"
cps_df

,year,month,inctot,date
0,1962,1,0.0,1962-1-1
3,1962,1,1692.0,1962-1-1
4,1962,1,1522.0,1962-1-1
5,1962,1,1500.0,1962-1-1
6,1962,1,2300.0,1962-1-1
...,...,...,...,...
9957416,2020,1,9000.0,2020-1-1
9957417,2020,1,0.0,2020-1-1
9957418,2020,1,3187.0,2020-1-1
9957419,2020,1,0.0,2020-1-1


In [25]:
grp_df = cps_df.groupby(
    by = ["date"]
)["inctot"]

stat_df = pd.DataFrame()
stat_df["count"] =  grp_df.count()
stat_df["sum"] =  grp_df.sum()
stat_df["median"] =  grp_df.median()
stat_df["mean"] =  grp_df.mean()

stat_df.head()

# cps_df["timestamp"] = cps_df.apply(lambda row: dt.datetime(
#     row["year"].astype(int),
#     row["month"].astype(int),
#     1,
# ), axis = 1)

,count,sum,median,avg
date,,,,
1962-1-1,53508,120440806.0,500.0,2250.893436
1963-1-1,35687,82304251.0,500.0,2306.281027
1964-1-1,54530,134298022.0,600.0,2462.828205
1965-1-1,54494,140806234.0,669.5,2583.885088
1966-1-1,110034,298278451.0,700.0,2710.784403
